## Generate expected results

This notebook is used to run a specific version of the model and save each results dataframe as a csv. These are used in `test_backtest.py` to verify that the model produces consistent results.

The notebook is provided as it is possible that results may change due to alterations to the model structure and operations. Once it has been confirmed that changes are intentional and not any introduced errors, this script can be run to regenerate the `.csv` files used in the test.

The run time is provided at the end of this notebook.

## Set-up

Load notebook linters.

In [1]:
%load_ext pycodestyle_magic

In [2]:
%pycodestyle_on

Load required packages.

In [3]:
# To ensure any updates to `simulation/` are fetched without needing to restart
# the notebook environment, reload `simulation/` before execution of each cell
%load_ext autoreload
%autoreload 1
%aimport simulation

In [4]:
# Import required packages
from simulation.model import Defaults, Trial
import os
import time

Start timer.

In [5]:
# Start timer
start_time = time.time()

Define path to expected results.

In [6]:
# Define path to folder for expected results for tests
tests = '../tests/exp_results/'

## Run model and save results

In [7]:
# Define model parameters
param = Defaults()
param.patient_inter = 4
param.mean_n_consult_time = 10
param.number_of_nurses = 4
param.warm_up_period = 500
param.data_collection_period = 1500
param.number_of_runs = 5
param.audit_interval = 50
param.scenario_name = 0
param.cores = 1

# Run the trial
trial = Trial(param)
trial.run_trial()

In [8]:
# Patient-level results
display(trial.patient_results_df)
trial.patient_results_df.write_csv(os.path.join(tests, 'patient.csv'))

patient_id,arrival_time,q_time_nurse,time_with_nurse,run
i64,f64,f64,f64,i32
1,504.513444,0.0,29.462508,0
2,507.230909,0.0,7.345594,0
3,508.394374,0.0,7.986073,0
4,512.975372,0.0,7.127312,0
5,521.933116,0.0,6.654871,0
…,…,…,…,…
365,1978.104484,0.0,11.023619,4
366,1980.171724,0.0,8.212316,4
367,1986.506388,0.0,24.89816,4


In [9]:
# Trial-level results
display(trial.trial_results_df)
trial.trial_results_df.write_csv(os.path.join(tests, 'trial.csv'))

run_number,scenario,arrivals,mean_q_time_nurse,mean_time_with_nurse,mean_nurse_utilisation
i64,i64,i64,f64,f64,f64
0,0,403,2.065396,9.622971,0.643006
1,0,382,1.396755,10.186235,0.64564
2,0,369,3.484284,10.733822,0.66079
3,0,367,2.050522,10.511447,0.634541
4,0,369,1.836706,9.742127,0.597097


In [10]:
# Interval audit results
display(trial.interval_audit_df)
trial.interval_audit_df.write_csv(os.path.join(tests, 'interval.csv'))

resource_name,simulation_time,utilisation,queue_length,running_mean_wait_time,run
str,i64,f64,i64,f64,i32
"""nurse""",500,0.0,0,0.651192,0
"""nurse""",550,0.0,0,0.599917,0
"""nurse""",600,0.0,0,0.556127,0
"""nurse""",650,1.0,0,0.514799,0
"""nurse""",700,1.0,2,1.912841,0
…,…,…,…,…,…
"""nurse""",1750,1.0,1,2.365482,4
"""nurse""",1800,0.75,0,2.305149,4
"""nurse""",1850,1.0,2,2.248533,4


In [11]:
# Overall results
display(trial.overall_results_df)
trial.overall_results_df.write_csv(os.path.join(tests, 'overall.csv'))

metric,arrivals,mean_q_time_nurse,mean_time_with_nurse,mean_nurse_utilisation
str,f64,f64,f64,f64
"""mean""",378.0,2.166733,10.15932,0.636215
"""std_dev""",15.198684,0.784458,0.478668,0.023832
"""lower_95_ci""",359.128341,1.192699,9.564976,0.606623
"""upper_95_ci""",396.871659,3.140766,10.753665,0.665806


In [12]:
# Get run time in seconds
end_time = time.time()
runtime = round(end_time - start_time)

# Display converted to minutes and seconds
print(f'Notebook run time: {runtime // 60}m {runtime % 60}s')

Notebook run time: 0m 0s
